In [33]:
import pandas as pd

# Load your traffic data here (replace 'data.csv' with your filename)
traffic_data = pd.read_csv('data_for_poisson_regression.csv')

# Select relevant features (e.g., time, weather, road conditions) as explanatory variables (X)
# and number of vehicles as the response variable (y)
X = traffic_data[['months_passed', 'posts_in_month', 'posts_total']]
y = traffic_data['views']

In [34]:
X.isna().any().any()

False

In [35]:
np.log(X)

/home/ashish/anaconda3/lib/python3.9/site-packages/pandas/core/internals/blocks.py:402: RuntimeWarning: divide by zero encountered in log
  result = func(self.values, **kwargs)


,months_passed,posts_in_month,posts_total
0,0.000000,-inf,-inf
1,0.693147,1.386294,1.386294
2,1.098612,0.693147,1.791759
3,1.386294,-inf,1.791759
4,1.609438,-inf,1.791759
...,...,...,...
85,4.454347,2.197225,7.628518
86,4.465908,3.295837,7.641564
87,4.477337,3.637586,7.659643
88,4.488636,2.944439,7.668561


In [36]:
import statsmodels.api as sm
import numpy as np

# Check for NaN values
if X.isna().any().any():
  # Impute missing values with the mean (replace with your preferred method)
  X = X.fillna(X.mean())

logarithms = np.log(X)

# Filter Option: Using boolean indexing with isinf()
# logarithms_filtered = logarithms[~logarithms.isinf().any(axis=1)]

# Define the Poisson regression model

# checking for infinite values and displaying the count
count = np.isinf(logarithms).values.sum()
print(count)


# Option 1: Using isinf().any() with list comprehension
logarithms_filtered = logarithms[logarithms['posts_in_month'] != -np.inf]


8


/home/ashish/anaconda3/lib/python3.9/site-packages/pandas/core/internals/blocks.py:402: RuntimeWarning: divide by zero encountered in log
  result = func(self.values, **kwargs)


In [37]:
logarithms

,months_passed,posts_in_month,posts_total
0,0.000000,-inf,-inf
1,0.693147,1.386294,1.386294
2,1.098612,0.693147,1.791759
3,1.386294,-inf,1.791759
4,1.609438,-inf,1.791759
...,...,...,...
85,4.454347,2.197225,7.628518
86,4.465908,3.295837,7.641564
87,4.477337,3.637586,7.659643
88,4.488636,2.944439,7.668561


In [38]:
logarithms_filtered

,months_passed,posts_in_month,posts_total
1,0.693147,1.386294,1.386294
2,1.098612,0.693147,1.791759
7,2.079442,0.000000,1.945910
8,2.197225,0.693147,2.197225
9,2.302585,0.693147,2.397895
...,...,...,...
85,4.454347,2.197225,7.628518
86,4.465908,3.295837,7.641564
87,4.477337,3.637586,7.659643
88,4.488636,2.944439,7.668561


In [39]:
y_filtered = y.loc[logarithms_filtered.index]

In [40]:
model = sm.Poisson(y_filtered, logarithms_filtered)  # Log link function for Poisson regression

# Fit the model
model_fit = model.fit()

# Print the summary of the fitted model
print(model_fit.summary())

Optimization terminated successfully.
         Current function value: nan
         Iterations 2
                          Poisson Regression Results                          
Dep. Variable:                  views   No. Observations:                   83
Model:                        Poisson   Df Residuals:                       80
Method:                           MLE   Df Model:                            2
Date:                Fri, 12 Apr 2024   Pseudo R-squ.:                     nan
Time:                        18:27:09   Log-Likelihood:                    nan
converged:                       True   LL-Null:                   -5.1255e+05
Covariance Type:            nonrobust   LLR p-value:                       nan
                     coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------------------------------
months_passed         nan        nan        nan        nan         nan         nan
posts_in_month        

/home/ashish/anaconda3/lib/python3.9/site-packages/statsmodels/discrete/discrete_model.py:1009: RuntimeWarning: overflow encountered in exp
  return stats.poisson.cdf(y, np.exp(X))
/home/ashish/anaconda3/lib/python3.9/site-packages/statsmodels/discrete/discrete_model.py:1333: RuntimeWarning: overflow encountered in exp
  L = np.exp(np.dot(X,params) + exposure + offset)
/home/ashish/anaconda3/lib/python3.9/site-packages/statsmodels/discrete/discrete_model.py:1243: RuntimeWarning: overflow encountered in exp
  L = np.exp(np.dot(X,params) + offset + exposure)


In [45]:
# Create a new data point with desired feature values for prediction
new_data = pd.DataFrame({'months_passed': [91],
                          'posts_in_month': [7],
                          'posts_total': [2156]})

new_data_logarithms = np.log(new_data)

# Predict the expected traffic volume based on the fitted model
predicted_traffic = model_fit.predict(new_data_logarithms)

# Round the predicted value (as it represents expected count)
print("Predicted traffic volume:", round(predicted_traffic))


Predicted traffic volume: 0   NaN
dtype: float64
